<a href="https://colab.research.google.com/github/rehmanaly0051/Exterior-Design-Model/blob/main/exterior_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import torch
torch.__version__

'2.6.0+cu124'

In [ ]:
import torch

print(torch.cuda.is_available())
if torch.cuda.is_available():
  print(torch.cuda.get_device_name(0))
else:
  print('NotAvailable')

True
Tesla T4


In [ ]:
import torch
print(torch.version.cuda)

12.4


In [ ]:
#!pip install -U xformers --index-url https://download.pytorch.org/whl/cu124

In [ ]:
%cd "/content/drive/MyDrive/Exterior Design"

/content/drive/MyDrive/Exterior Design


In [ ]:
import os
print("Working dir:", os.getcwd())
print("Exists?", os.path.exists(floor_plan_path))

Working dir: /content/drive/MyDrive/Exterior Design
Exists? True


In [ ]:
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
import numpy as np
from PIL import Image
import cv2
import os

def preprocess_floor_plan(floor_plan_path, output_size=(512, 512)):
    """Process floor plan for better edge detection"""
    # Load image
    image = cv2.imread(floor_plan_path)
    if image is None:
        raise ValueError(f"Could not load image from {floor_plan_path}")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize while maintaining aspect ratio
    h, w = image.shape[:2]
    ratio = min(output_size[0] / w, output_size[1] / h)
    new_size = (int(w * ratio), int(h * ratio))
    resized = cv2.resize(image, new_size)

    # Create white canvas of target size
    canvas = np.ones((output_size[1], output_size[0], 3), dtype=np.uint8) * 255

    # Place the resized image on the canvas
    offset_x = (output_size[0] - new_size[0]) // 2
    offset_y = (output_size[1] - new_size[1]) // 2
    canvas[offset_y:offset_y+new_size[1], offset_x:offset_x+new_size[0]] = resized

    # Convert to grayscale for edge detection
    gray = cv2.cvtColor(canvas, cv2.COLOR_RGB2GRAY)

    # Adjust threshold parameters for better edge detection
    # These parameters work well for floor plans which typically have clear lines
    edges = cv2.Canny(gray, 100, 200)

    # Dilate edges to make them more prominent
    kernel = np.ones((2, 2), np.uint8)
    dilated_edges = cv2.dilate(edges, kernel, iterations=1)

    # Convert back to RGB for ControlNet
    edge_image = cv2.cvtColor(dilated_edges, cv2.COLOR_GRAY2RGB)

    # Save the processed floor plan for debugging/reference
    Image.fromarray(edge_image).save("processed_floor_plan.png")

    return Image.fromarray(edge_image)

def generate_exterior_design(
    floor_plan_path,
    output_dir="exterior_designs",
    architectural_style="modern",
    view_type="front",
    resolution=(768, 768),
    num_samples=1
):

    # Create output directory
    os.makedirs(output_dir, exist_ok=True)

    # Load ControlNet model
    controlnet = ControlNetModel.from_pretrained(
        "lllyasviel/sd-controlnet-canny",
        torch_dtype=torch.float16
    )

    # Create pipeline with a model optimized for architectural images
    # We use "stable-diffusion-2-1" which has better architectural understanding
    pipe = StableDiffusionControlNetPipeline.from_pretrained(
        "stabilityai/stable-diffusion-2-1",
        controlnet=controlnet,
        torch_dtype=torch.float16
    )

    # Use more efficient scheduler
    pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

    # Move to GPU if available
    device = "cuda" if torch.cuda.is_available() else "cpu"
    pipe = pipe.to(device)

    # Enable memory optimization if on GPU
    if device == "cuda":
        pipe.enable_xformers_memory_efficient_attention()

    # Process the floor plan for edge detection
    control_image = preprocess_floor_plan(floor_plan_path)

    # Build the prompt for the specific style and view
    style_descriptors = {
        "modern": "modern minimalist architecture, flat roof, large windows, clean lines",
        "traditional": "traditional architecture, sloped roof, symmetrical facade, classic design",
        "contemporary": "contemporary architecture, mixed materials, geometric shapes, innovative design",
        "mediterranean": "mediterranean villa style, terracotta roof tiles, stucco walls, arched features",
        "farmhouse": "modern farmhouse exterior, gabled roof, porch, board and batten siding",
        "industrial": "industrial architecture, exposed metal, large windows, raw materials"
    }

    view_descriptors = {
        "front": "front elevation of a house",
        "side": "side elevation of a house",
        "aerial": "aerial view of a house",
        "perspective": "3/4 perspective view of a house exterior"
    }

    # Get style and view descriptions, with fallbacks to default values
    style_desc = style_descriptors.get(architectural_style.lower(), style_descriptors["modern"])
    view_desc = view_descriptors.get(view_type.lower(), view_descriptors["front"])

    # Build the final prompt
    prompt = f"{view_desc}, {style_desc}, high-quality architectural visualization, exterior design rendering, high detail, photorealistic, professional architectural photography"

    # Negative prompt to avoid common issues
    negative_prompt = "interior, floor plan, blueprint, sketch, drawing, wireframe, cutaway, cross-section, distorted, blurry, deformed, disfigured, poorly drawn, extra limbs, text, watermark, signature, cut off, low quality"

    # Generate images
    generated_paths = []
    for i in range(num_samples):
        print(f"Generating exterior design {i+1}/{num_samples}...")
        images = pipe(
            prompt,
            control_image,
            negative_prompt=negative_prompt,
            num_inference_steps=30,
            guidance_scale=7.5,
            width=resolution[0],
            height=resolution[1]
        ).images

        # Save the image
        for j, image in enumerate(images):
            output_path = os.path.join(output_dir, f"{architectural_style}_{view_type}_{i+1}_{j+1}.png")
            image.save(output_path)
            generated_paths.append(output_path)
            print(f"Saved to {output_path}")

    return generated_paths

def generate_multiple_exterior_views(
    floor_plan_path,
    output_dir="exterior_designs",
    architectural_style="modern",
    resolution=(768, 768)
):
    """Generate multiple views of the exterior design"""
    view_types = ["front", "side", "perspective", "aerial"]
    all_generated_paths = []

    for view_type in view_types:
        print(f"\nGenerating {view_type} view...")
        generated_paths = generate_exterior_design(
            floor_plan_path=floor_plan_path,
            output_dir=output_dir,
            architectural_style=architectural_style,
            view_type=view_type,
            resolution=resolution,
            num_samples=1  # Generate one sample per view type
        )
        all_generated_paths.extend(generated_paths)

    print(f"\nAll exterior designs saved to {output_dir}")
    return all_generated_paths

def generate_multiple_styles(
    floor_plan_path,
    output_dir="exterior_designs",
    view_type="front",
    resolution=(768, 768)
):
    """Generate exterior designs in multiple architectural styles"""
    styles = ["modern", "traditional", "contemporary", "mediterranean", "farmhouse", "industrial"]
    all_generated_paths = []

    for style in styles:
        print(f"\nGenerating {style} style...")
        generated_paths = generate_exterior_design(
            floor_plan_path=floor_plan_path,
            output_dir=output_dir,
            architectural_style=style,
            view_type=view_type,
            resolution=resolution,
            num_samples=1  # Generate one sample per style
        )
        all_generated_paths.extend(generated_paths)

    print(f"\nAll exterior designs saved to {output_dir}")
    return all_generated_paths

if __name__ == "__main__":
    # Example usage
    floor_plan_path = "/content/drive/MyDrive/Exterior Design/test_resized.png"  # Replace with your floor plan path

    # Option 1: Generate one specific exterior design
    generate_exterior_design(
        floor_plan_path=floor_plan_path,
        architectural_style="modern",
        view_type="front"
    )

    # Option 2: Generate multiple views of the same style
    '''generate_multiple_exterior_views(
        floor_plan_path=floor_plan_path,
        architectural_style="contemporary"
    )'''

    # Option 3: Generate the same view in multiple styles
    '''generate_multiple_styles(
        floor_plan_path=floor_plan_path,
        view_type="perspective"
    )'''

Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

UnboundLocalError: cannot access local variable 'edge_image' where it is not associated with a value